In [1]:
import os
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Activation, Dropout, Maximum, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.applications.resnet50 import preprocess_input
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam, SGD
from keras import Model
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.utils import class_weight
import seaborn as sn
from keras.layers import LeakyReLU
import pandas as pd

Using TensorFlow backend.


In [2]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
batch_size = 64

CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

# Same as id_to_label_dict

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

train_dir = "D:/Neural_nets_course_Dataset/all/train/"
test_dir = "D:/Neural_nets_course_Dataset/all/test/"

model_check_point_loc = 'D:/Neural_nets_course_Dataset/all/models/resnet_50/resnet_50_model.h5'

resnet_base = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))


In [3]:
# Freeze the layers except the last 2 layers
for layer in resnet_base.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in resnet_base.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x000001AB4DD4DA90> False
<keras.layers.convolutional.ZeroPadding2D object at 0x000001AB4DD4DEF0> False
<keras.layers.convolutional.Conv2D object at 0x000001AB4DD4DD68> False
<keras.layers.normalization.BatchNormalization object at 0x000001AB4DD4DDA0> False
<keras.layers.core.Activation object at 0x000001AB4DD73B00> False
<keras.layers.pooling.MaxPooling2D object at 0x000001AB4DD9CD68> False
<keras.layers.convolutional.Conv2D object at 0x000001AB4DD65EF0> False
<keras.layers.normalization.BatchNormalization object at 0x000001AB53551160> False
<keras.layers.core.Activation object at 0x000001AB4DF2E7F0> False
<keras.layers.convolutional.Conv2D object at 0x000001AB53579A20> False
<keras.layers.normalization.BatchNormalization object at 0x000001AB53566CC0> False
<keras.layers.core.Activation object at 0x000001AB535C5550> False
<keras.layers.convolutional.Conv2D object at 0x000001AB53A6B160> False
<keras.layers.convolutional.Conv2D object at 0x000

In [4]:
x = resnet_base.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=resnet_base.input, outputs=predictions)
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
class_labels = os.listdir(train_dir)

In [6]:
def generate_XY(size=224):
    combined_feature_list = []
    labels = []

    for idx, level in enumerate(class_labels):
        retina_root = train_dir + level + '/'
        retina_root_list = os.listdir(retina_root)

        for index, relative_path in enumerate(retina_root_list):
            if relative_path == 'Thumbs.db':
                continue
            source = retina_root + relative_path
            img = image.load_img(source, target_size=(image_width, image_height))
            img_x = image.img_to_array(img)
            #img_x = np.expand_dims(img_x, axis=0)
            #resnet50_x = preprocess_input(img_x)
            
            combined_feature_list.append(img_x)
            labels.append(int(level))
            
    return np.array(combined_feature_list), np.array(labels)
    

In [7]:
X_train, y_train = generate_XY()

In [8]:
y_train_onehot = to_categorical(y_train, 12)

In [9]:
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train)
                                               ,y_train)

In [10]:
dict(enumerate(class_weight))

{0: 1.5050697084917617,
 1: 1.014957264957265,
 2: 1.3792102206736354,
 3: 0.6478450627386798,
 4: 1.7911010558069382,
 5: 0.8333333333333334,
 6: 0.605249745158002,
 7: 1.7911010558069382,
 8: 0.7671188630490956,
 9: 1.7135642135642135,
 10: 0.7980510752688172,
 11: 1.0281385281385282}

In [11]:
class_weight_dic = dict(enumerate(class_weight))

In [12]:
X_train.shape[1:]

(224, 224, 3)

In [13]:
X = X_train
#X = transformed_data
Y = y_train

In [14]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=30)
#kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=25)
cvscores = []
trainScores = []
f1Score = []
num_k_folds = 5
fold_counter = 0
val_conmats = []
val_precisions = []
val_recalls = []
val_f_scores = []

In [15]:
for train, test in kfold.split(X, Y):
    # Fit the model
    model.fit(X[train], to_categorical(Y[train]), epochs=30, batch_size=batch_size, class_weight=class_weight_dic, verbose=0)
    
    y_train_pred = model.predict(X[train])
    y_train_pred = np.argmax(y_train_pred, axis=1)
    
    y_validation_pred = model.predict(X[test])
    y_validation_pred = np.argmax(y_validation_pred, axis=1)
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    
    [precision, recall, f_score, _] = precision_recall_fscore_support(Y[test], y_validation_pred)
    #print("Validation k-fold #%d - precision: %f, recallL: %f, f-score: %f" % (fold_counter, precision, recall, f_score))
    
    conmat = confusion_matrix(Y[test], y_validation_pred)
    
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_f_scores.append(f_score)
    val_conmats.append(conmat)
    fold_counter = fold_counter + 1
    
    trainScores.append(accuracy_score(Y[train], y_train_pred))
    cvscores.append(accuracy_score(Y[test], y_validation_pred))
    
print("\nAveraging the 5-fold results:")
print("%s: %.2f%%" % ('AVG Train Acc ', np.mean(trainScores) * 100))
print("%s: %.2f%%" % ('AVG Validation Acc ', np.mean(cvscores) * 100))
print("Validation precision - mean: %f, stddev: %f" % (np.mean(val_precisions), np.std(val_precisions)))
print("Validation recall - mean: %f, stddev: %f" % (np.mean(val_recalls), np.std(val_recalls)))
print("Validation f-score - mean: %f, stddev: %f" % (np.mean(val_f_scores), np.std(val_f_scores)))
print("Confusion matrix:")
print (sum(val_conmats).astype(float) / fold_counter)

KeyboardInterrupt: 

In [ ]:
confusion_matrix_array = (sum(val_conmats).astype(float) / fold_counter)
new_confusion_matrix_array = []
for conf_sub_array in confusion_matrix_array:
    new_confusion_matrix_array.append([int(float(e)) for e in conf_sub_array])
        
df_cm = pd.DataFrame(new_confusion_matrix_array, range(12), range(12))
#plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 12})# font size

In [ ]:
test_image_list = os.listdir(test_dir)

In [ ]:
def generate_testXY(size=224):
    test_feature_list = []
    label_names = []

    for index, relative_path in enumerate(test_image_list):
        if relative_path == 'Thumbs.db':
            continue
        source = test_dir + relative_path
        img = image.load_img(source, target_size=(image_width, image_height))
        img_x = image.img_to_array(img)
        #img_x = np.expand_dims(img_x, axis=0)
        #resnet50_x = preprocess_input(img_x)

        label_names.append(relative_path)
        test_feature_list.append(img_x)
        #labels.append(int(level))
            
    return np.array(test_feature_list), np.array(label_names)

In [ ]:
X_test, label_list = generate_testXY()

In [ ]:
# Predictions
prob = model.predict(X_test, verbose=1)
pred = prob.argmax(axis=-1)

submission_df = pd.DataFrame({
    "file": [label for label in label_list],
    "species": [INV_CLASS[p] for p in pred]
})

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('D:/Neural_nets_course_Dataset/all/output/custom/submission_resnet.csv', index=False, header=True)